In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("hw12.ipynb")

<img style="display: block; margin-left: auto; margin-right: auto" src="./ccsf-logo.png" width="250rem;" alt="The CCSF black and white logo">

# Homework 12: Error Analysis

## References

* [Inference for Regression](https://inferentialthinking.com/chapters/16/Inference_for_Regression.html)
* [`datascience` Documentation](https://datascience.readthedocs.io/)
* [Python Quick Reference](https://ccsf-math-108.github.io/materials-sp24/resources/quick-reference.html)

## Assignment Reminders

- Make sure to run the code cell at the top of this notebook that starts with `# Initialize Otter` to load the auto-grader.
- For all tasks indicated with a 🔎 that you must write explanations and sentences for, provide your answer in the designated space.
- Throughout this assignment and all future ones, please be sure to not re-assign variables throughout the notebook! _For example, if you use `max_temperature` in your answer to one question, do not reassign it later on. Otherwise, you will fail tests that you thought you were passing previously!_
- We encourage you to discuss this assignment with others but make sure to write and submit your own code. Refer to the syllabus to learn more about how to learn cooperatively.
- Unless you are asked otherwise, use the non-interactive visualizations when asked to produce a visualization for a task.
- View the related <a href="https://ccsf.instructure.com" target="_blank">Canvas</a> Assignment page for additional details.

Run the following code cell to import the tools for this assignment.

In [ ]:
from datascience import *
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

import warnings
warnings.simplefilter('ignore')

## Regression Inference

Previously, you've used confidence intervals to quantify uncertainty about estimates. You can also run hypothesis tests using a confidence interval under the following procedure:

1. Define a null and alternative hypothesis (they must be of the form "The parameter is X" and "The parameter is not X").
2. Choose a p-value cutoff, and call it $q$.
3. Construct a $(100-q)\%$ interval using bootstrap sampling (for example, if your p-value cutoff is 0.01, or 1%, then construct a 99% confidence interval).
4. Using the confidence interval, determine if your data are more consistent with your null or alternative hypothesis:
   * If the null hypothesis parameter X is in your confidence interval, the data are more consistent with the null hypothesis.
   * If the null hypothesis parameter X is *not* in your confidence interval, the data are more consistent with the alternative hypothesis.

More recently, we've discussed the use of linear regression to make predictions based on correlated variables. For example, we can predict the height of children based on the heights of their parents.

We can combine these two topics to make powerful statements about our population by using the following techniques:

- Bootstrapped interval for the true slope
- Bootstrapped prediction interval for y (given a particular value of x)

This homework explores these two methods.

## The Data

The [Snowy Plover](https://www.audubon.org/field-guide/bird/snowy-plover) is a tiny bird that lives on the coast in parts of California and elsewhere. It is so small that it is vulnerable to many predators, including people and dogs that don't look where they are stepping when they go to the beach. It is considered endangered in many parts of the U.S.

The data are about the eggs and newly-hatched chicks of the Snowy Plover. Here's a picture of [a parent bird incubating its eggs](http://cescos.fau.edu/jay/eps/articles/snowyplover.html).

<img src="plover_and_eggs.jpeg" alt="Plover and Eggs">

The data were collected at the Point Reyes National Seashore by a former [student at Berkeley](https://openlibrary.org/books/OL2038693M/BLSS_the_Berkeley_interactive_statistical_system). The goal was to see how the size of an egg could be used to predict the weight of the resulting chick. The bigger the newly-hatched chick, the more likely it is to survive.

<img src="plover_and_chick.jpeg" alt="Plover and Chick">

Each row of the table below corresponds to one Snowy Plover egg and the resulting chick. Note how tiny the bird is:

- `Egg Length` and `Egg Breadth` (widest diameter) are measured in millimeters
- `Egg Weight` and `Bird Weight` are measured in grams; for comparison, a standard paper clip weighs about one gram

In [ ]:
birds = Table.read_table('snowy_plover.csv')
birds

In this investigation, we will be using the egg weight to predict bird weight. Run the cell below to create a scatter plot of the egg weights and bird weights, along with their line of best fit.

In [ ]:
# Just run this cell and examine the scatter plot.
birds.scatter('Egg Weight', "Bird Weight", fit_line=True)

## Residual Plot

It seems like a linear model might be appropriate to apply to this situation, but you should check the residual plot before moving forward. You'll likely find the following functions useful.

In [ ]:
def standard_units(data):
    return (data - np.mean(data)) / np.std(data)

def correlation(tbl, x_col, y_col):
    return np.mean(standard_units(tbl.column(x_col)) * standard_units(tbl.column(y_col)))

#### Task 01 📍

Create a function called `fit_line`.  It should take a table (e.g. `birds`) and the column names associated with *x* and *y* as its arguments and return an *array* containing the slope and intercept of the regression line (in that order) that predicts the *y* column in the table using the *x* column.

_Points:_ 3

In [ ]:
def fit_line(tbl, x_col, y_col):
    r = ...
    slope = ...
    intercept = ...
    return make_array(slope, intercept)

fit_line(birds, "Egg Weight", "Bird Weight")

In [ ]:
grader.check("task_01")

Now, run the following cell to plot the line produced by calling `fit_line` on the `birds` table.

**Note:** You are not responsible for the code in the cell below, but make sure that your `fit_line` function generates a reasonable line for the data. Compare the line with the line produced in the scatter plot above with the `fit_line=True` parameter.

In [ ]:
# Just run this cell
slope, intercept = fit_line(birds, "Egg Weight", "Bird Weight")
birds.scatter("Egg Weight", "Bird Weight")
plt.plot([min(birds.column("Egg Weight")), 
          max(birds.column("Egg Weight"))], 
         [slope*min(birds.column("Egg Weight"))+intercept, 
          slope*max(birds.column("Egg Weight"))+intercept])
plt.show()

### Task 02 📍

With a linear regression model, update the `birds` table with a column of predicted birth weights using the regression line and the associated residuals.

1. Create an array called `predicted_bird_weights` that contains the predicted bird weights from the regression line.
2. Create an array called `residuals` that contains the residuals associated with those predictions.
3. Update the `birds` table to include columns `'Predicted Bird Weight'` and `'Residual'` with their respective data.

_Points:_ 3

In [ ]:
predicted_bird_weights = ...
residuals = ...
birds = birds.with_columns(
    'Predicted Bird Weight', ...,
    'Residual', ...)

In [ ]:
grader.check("task_02")

### Task 03 📍🔎

<!-- BEGIN QUESTION -->

Now that you have the residuals for these predictions:
1. Create a residual plot showing egg weights on the horizontal axis and the residuals on the vertical axis.
2. In the space provided, provide a short justification as to why it is reasonable to predict bird weight from egg weight using the linear model. Make sure to point out features from the residual plot relevant to [Section 15.5](https://inferentialthinking.com/chapters/15/5/Visual_Diagnostics.html#regression-diagnostics) that back up your justification. Additionally, it is reasonable to express concerns about applying a model to a given situation, so you are welcome to point out any concerns in relation to the content from Section 15.5.

_Points:_ 3

_Type your answer here, replacing this text._

In [ ]:
# Generate the residual plot here
birds.scatter('Egg Weight', 'Residual')

<!-- END QUESTION -->

## Finding the Bootstrap Confidence Interval for the True Slope

In [ ]:
# Just run this cell
birds.scatter('Egg Weight', "Bird Weight", fit_line=True)

Looking at the scatter plot of our sample, we observe a linear relationship between egg weight and bird weight. However, relationships that appear in a sample might not exist in the population from which the sample was taken.

We want to know if there truly exists a linear relationship between egg weight and bird weight for Snowy Plovers. If there is no linear relationship between the two variables, then we'd expect a correlation of 0. Consequently, the slope of the regression line would also be 0. 

We highly recommend reviewing section [16.2](https://inferentialthinking.com/chapters/16/2/Inference_for_the_True_Slope.html) before attempting this part of the homework

#### Task 04 📍🔎

<!-- BEGIN QUESTION -->

Let's run a hypothesis test using confidence intervals to see if there is a linear relationship between egg weight and bird weight. Define the null and alternative hypotheses that will allow you to conduct this test.

*Note:* Please write your answer **in the cell below** in the following format:
- **Null Hypothesis:**
- **Alternative Hypothesis:**

_Points:_ 2

_Type your answer here, replacing this text._

<!-- END QUESTION -->

Next, you will create a confidence interval that quantifies our uncertainty about the true relationship between egg weight and bird weight.

#### Task 05 📍🔎

<!-- BEGIN QUESTION -->

Create an array called `resampled_slopes` that contains the slope of the best fit line for 1,000 bootstrap resamples of `birds`. Plot the distribution of these slopes.

_Points:_ 2

In [ ]:
resampled_slopes = ...

for i in np.arange(1_000): 
    birds_bootstrap = ...
    bootstrap_line = ...
    bootstrap_slope = ...
    resampled_slopes = ...
    
# DO NOT CHANGE THIS LINE
Table().with_column("Slope estimate", resampled_slopes).hist()

<!-- END QUESTION -->

#### Task 06 📍

Use your resampled slopes to construct an 95% confidence interval for the true value of the slope.

_Points:_ 3

In [ ]:
lower_end = ...
upper_end = ...
print("95% confidence interval for slope: [{:g}, {:g}]".format(lower_end, upper_end))

In [ ]:
grader.check("task_06")

#### Task 07 📍🔎

<!-- BEGIN QUESTION -->

In your response, make to address the following:
1. Based on your confidence interval, would you **reject** or **fail to reject** the null hypothesis that the true slope is 0? 
2. Provide an explanation Why?  
3. What p-value cutoff are you using?

**Hint:** Re-read the first section of this homework!

_Points:_ 2

_Type your answer here, replacing this text._

<!-- END QUESTION -->

## Finding the Bootstrap Prediction Interval

Suppose we're visiting Point Reyes and stumble upon some Snowy Plover eggs; we'd like to know how heavy they'll be once they hatch. In other words, we want to use our regression line to make predictions about a bird's weight based on the weight of the corresponding egg.

However, just as we're uncertain about the slope of the true regression line, we're also uncertain about the predictions made based on the true regression line.

#### Task 08 📍

Define the function `fitted_value`.  It should take in four arguments:

1. `table`: a table like `birds`.  We'll be predicting the values in the second column using the first.
2. `x_col`: the name of our x-column within the input `table`
3. `y_col`: the name of our y-column within the input `table`
2. `given_x`: a number, the value of the predictor variable for which we'd like to make a prediction.
  
The function should return the line's prediction for the given x.

*Hint:* Make sure to use the `fit_line` function you defined in Task 11.

_Points:_ 2

In [ ]:
def fitted_value(table, x_col, y_col, given_x):
    line = ...
    slope = ...
    intercept = ...
    ...

# Here's an example of how fitted_value is used. The code below
# computes the prediction for the bird weight, in grams, based on
# an egg weight of 8 grams.
egg_weight_eight = fitted_value(birds, "Egg Weight", "Bird Weight", 8)
egg_weight_eight

In [ ]:
grader.check("task_08")

#### Task 09 📍

Raymond, the resident Snowy Plover expert at Point Reyes, tells us that the egg he has been carefully observing has a weight of 9 grams. Using `fitted_value` above, assign the variable `experts_egg` to the predicted bird weight for Raymond's egg.

_Points:_ 2

In [ ]:
experts_egg = ...
experts_egg

In [ ]:
grader.check("task_09")

Let's look at the number of rows in the birds table.

In [ ]:
birds.num_rows

A fellow parkgoer raises the following objection to your prediction:

> "Your prediction depends on your sample of 44 birds.  Wouldn't your prediction change if you had a different sample of 44 birds?"

Having read section [16.3](https://inferentialthinking.com/chapters/16/3/Prediction_Intervals.html) of the textbook, you know just the response! Had the sample been different, the regression line would have been different too. This would ultimately result in a different prediction. To see how good our prediction is, we must get a sense of how variable the prediction can be.

#### Task 10 📍

Define a function `compute_resampled_line` that takes in a table `tbl`and two column names, `x_col` and `y_col`, and returns an array containing the parameters of the best fit line (slope and intercept) for one bootstrapped resample of the table.

_Points:_ 2

In [ ]:
def compute_resampled_line(tbl, x_col, y_col):
    resample = ...
    resampled_line = ...
    ...

In [ ]:
grader.check("task_10")

**Run** the following cell below in order to define the function `bootstrap_lines`.  It takes in four arguments:
1. `tbl`: a table like `birds`
2. `x_col`: the name of our x-column within the input `tbl`
3. `y_col`: the name of our y-column within the input `tbl`
4. `num_bootstraps`: an integer, a number of bootstraps to run.

It returns a *table* with one row for each bootstrap resample and the following two columns:
1. `Slope`: the bootstrapped slopes 
2. `Intercept`: the corresponding bootstrapped intercepts 

In [ ]:
# Just run this cell
def bootstrap_lines(tbl, x_col, y_col, num_bootstraps):
    resampled_slopes = make_array()
    resampled_intercepts = make_array() 
    for i in np.arange(num_bootstraps): 
        resampled_line = compute_resampled_line(tbl, x_col, y_col) 
        resampled_slope = resampled_line.item(0) 
        resampled_intercept = resampled_line.item(1) 
        resampled_slopes = np.append(resampled_slopes,resampled_slope)
        resampled_intercepts = np.append(resampled_intercepts,resampled_intercept)
    tbl_lines = Table().with_columns('Slope', resampled_slopes, 'Intercept', resampled_intercepts)
    return tbl_lines

regression_lines = bootstrap_lines(birds, "Egg Weight", "Bird Weight", 1000)
regression_lines

#### Task 11 📍🔎

<!-- BEGIN QUESTION -->

Create an array called `predictions_for_eight` that contains the predicted bird weights based on an egg of weight 8 grams for each regression line in `regression_lines`.

_Points:_ 2

In [ ]:
predictions_for_eight = ...

# This will make a histogram of your predictions:
table_of_predictions = Table().with_column('Predictions at Egg Weight=8', predictions_for_eight)
table_of_predictions.hist('Predictions at Egg Weight=8', bins=20)

<!-- END QUESTION -->

#### Task 12 📍

Create an approximate 95% confidence interval for the predictions in `predictions_for_eight`.

_Points:_ 3

In [ ]:
lower_bound = ...
upper_bound = ...

print('95% Confidence interval for predictions for x=8: (', lower_bound,",", upper_bound, ')')

In [ ]:
grader.check("task_12")

#### Task 13 📍

Set `plover_statements` to an array of integer(s) that correspond to statement(s) that are true.

1. The 95% confidence interval covers 95% of the bird weights for eggs that had a weight of eight grams in the population of snowy plovers.

2. The 95% confidence interval quantifies the uncertainty in our estimate of what the true line would predict.

_Points:_ 2

In [ ]:
plover_statements = ...

In [ ]:
grader.check("task_13")

## Submit your Homework to Canvas

Once you have finished working on the homework tasks, prepare to submit your work in Canvas by completing the following steps.

1. In the related Canvas Assignment page, check the rubric to know how you will be scored for this assignment.
2. Double-check that you have run the code cell near the end of the notebook that contains the command `"grader.check_all()"`. This command will run all of the run tests on all your responses to the auto-graded tasks marked with 📍.
3. Double-check your responses to the manually graded tasks marked with 📍🔎.
3. Select the menu item "File" and "Save Notebook" in the notebook's Toolbar to save your work and create a specific checkpoint in the notebook's work history.
4. Select the menu items "File", "Download" in the notebook's Toolbar to download the notebook (.ipynb) file. 
5. In the related Canvas Assignment page, click Start Assignment or New Attempt to upload the downloaded .ipynb file.

**Keep in mind that the auto-grader does not always check for correctness. Sometimes it just checks for the format of your answer, so passing the auto-grader for a task does not mean you got the answer correct for that task.**

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()